<a href="https://colab.research.google.com/github/yf591/sd-model-merge-tool/blob/main/04_Merge_Model_Maker_Ver1_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3ソース（Huggin Face, Civitai, MyDrive）の2モデル単純マージ対応可
*   Hugging Face \* Hugging Face
*   Civitai \* Civitai
*   MyDrive \* MyDrive
*   Hugging Face \* Civitai
*   Hugging Face \* MyDrive
*   Civitai \* MyDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import output

# Hugging Face Hub, PyTorch, その他必要なライブラリをインストール
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # PyTorchを使用して深層学習モデルを操作します。CUDAバージョン（例: `cu118`）を指定
!pip install diffusers transformers accelerate # Stable Diffusionを扱うための主要ライブラリです。モデルのロードや画像生成の操作を簡素化
!pip install safetensors # 安全かつ軽量なモデル保存形式（`.safetensors`）をサポート
!pip install huggingface-hub # Hugging Face Hubからモデルをダウンロード・管理
!pip install opencv-python # 生成した画像の前処理や後処理に使用
!pip install numpy # 数値計算ライブラリで、モデルや画像の操作に使う
!pip install matplotlib # 生成された画像の可視化に使う
!pip install tqdm # プログレスバーの表示
!pip install optuna # ハイパーパラメータ最適化

output.clear()

In [ ]:
import os
import torch
from safetensors.torch import load_file, save_file
from transformers import AutoConfig, AutoModel
from huggingface_hub import hf_hub_download
import requests
from tqdm import tqdm

In [ ]:
from getpass import getpass
from google.colab import userdata

# Hugging Faceで取得したTokenをこちらに貼る(トークンを非表示で入力)
HF_TOKEN = getpass("Hugging FaceのRead権限のあるHF Tokenを入力してください: ")

# CIVITAI_TOKEN が存在する場合、取得
api_key = userdata.get('CIVITAI_TOKEN')
if api_key is None:
    print("Error: CIVITAI_API_KEY secret is not set.")

In [ ]:
#@title ### 関数の定義（設定）

# ヘルパー関数
def download_model(repo_id, filename, token):
    """Hugging Face Hubからモデルをダウンロード"""
    return hf_hub_download(repo_id=repo_id, filename=filename, token=token)


def download_civitai_model(url, output_path, api_key):
    """Civitaiからモデルをダウンロード"""
    try:
        headers = {"Authorization": f"Bearer {api_key}"} if api_key else None
        response = requests.get(url, stream=True, headers=headers)
        response.raise_for_status()

        total_size = int(response.headers.get('content-length', 0))
        with open(output_path, 'wb') as file, tqdm(
            desc=output_path,
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
             for data in response.iter_content(chunk_size=1024):
                size = file.write(data)
                bar.update(size)
        return output_path
    except Exception as e:
        print(f"Error downloading from Civitai: {e}")
        return None


def load_model(path, device):
    """ファイルパスからモデルをロードする"""
    try:
        if path.startswith("http"):
            # URLの場合（Civitaiなど）は、ダウンロードしてから読み込む
            if "civitai.com" in path:
                output_path = "/content/temp_model.safetensors"  # 一時的な保存先パス
                path = download_civitai_model(path, output_path, api_key)
                if not path:
                   return None
                else:
                   print(f"Civitaiからモデルをロード: {path}")
                   return load_file(path, device=device) #ダウンロード後のファイルパスを渡す。
            elif "huggingface.co" in path:
                print(f"HuggingFaceからモデルをロード: {path}")
                repo_id_and_file = path.split("huggingface.co/")[1]
                repo_id = repo_id_and_file.split("/resolve/")[0]
                filename = repo_id_and_file.split("/")[-1]
                path = download_model(repo_id, filename, HF_TOKEN)
                return load_file(path, device=device)
            else:
                print("Error: HTTP URL not recognized, use HuggingFace or Civitai Model.")
                return None

        if path.startswith("/content/drive"):
          # Google Drive のパスの場合
            print(f"Google Driveからモデルをロード: {path}")
            return load_file(path, device=device)
        else:
          print("Error: Incorrect Model Path.")
          return None
    except Exception as e:
        print(f"Error loading model: {e}")
        return None


def merge_weights(model1, model2, alpha):
    """指定されたレイヤーで重みをマージする"""
    merged = {}
    for key in model1.keys():
        if key in model2:
            layer_weight = alpha
            try:
                merged[key] = layer_weight * model1[key] + (1 - layer_weight) * model2[key]
            except Exception as e:
                print(f"Skipped key: {key}, Error: {e}")
                merged[key] = model1[key]  # エラーが発生したら model1 の重みをそのまま使う
        else:
            merged[key] = model1[key]
    return merged


def save_merged_model(merged_weights, output_path):
    """マージ済みモデルを保存"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    save_file(merged_weights, output_path)

**使い方**

*   **MyDriveにあるモデル:**
    `path1` に、Google Drive 内のモデルファイルのパスを指定します。
    
    例:`"/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/chilled_remix_v2.safetensors"`

*   **Hugging Face Hub のモデル:**
    `path1` または `path2` に、モデルのdownload linkを入力します。
    
    例:`"https://huggingface.co/casque/majicmixRealistic_v6/resolve/main/majicmixRealistic_v6.safetensors"`
    この場合、HuggingFaceからモデルをダウンロードします。

*   **Civitai のモデル:**
    `path1` または `path2` に、Civitai のモデルダウンロード URL を指定します。
    
    例:`"https://civitai.com/api/download/models/303526?type=Model&format=SafeTensor&size=full&fp=fp16?token={api_key}"`
    この場合、`CIVITAI_TOKEN` が設定されている必要があり、Civitaiからモデルをダウンロードします。

In [ ]:
#@title ### UI設定

#@markdown ### マージするモデルを設定
path1 = "https://huggingface.co/casque/majicmixRealistic_v6/resolve/main/majicmixRealistic_v6.safetensors" #@param {type:"string"}
path2 = "https://civitai.com/api/download/models/90505?type=Model&format=SafeTensor&size=full&fp=fp32"  #@param {type:"string"}

#@markdown ### アウトプット先を指定
output_file = "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/merged_model_sample.safetensors" #@param {type:"string"}

#@markdown ### マージ比率の設定
alpha = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}

In [ ]:
#@title ###実行
try:
    print("モデルのロードを開始します...")
    model1 = load_model(path1, device="cuda" if torch.cuda.is_available() else "cpu")
    model2 = load_model(path2, device="cuda" if torch.cuda.is_available() else "cpu")

    if model1 is None or model2 is None:
         print("モデルのロードに失敗しました。")
    else:

        # ここにモデルのキーを出力するコードを追加
        print("model1 keys:", list(model1.keys())[:5] , "...")
        print("model2 keys:", list(model2.keys())[:5] , "...")

        print("モデルをレイヤーごとにマージ中...")
        merged_weights = merge_weights(model1, model2, alpha)

        print(f"マージされたモデルを保存します: {output_file}")
        save_merged_model(merged_weights, output_file)
        print("マージ完了！")

except Exception as e:
    print(f"エラーが発生しました: {e}")